# https://github.com/apache/hbase/blob/master/hbase-thrift/src/main/resources/org/apache/hadoop/hbase/thrift2/hbase.thrift

In [8]:
import thrift as t
from hbase_thrift.hbase.ttypes import *

from thrift.transport import *
from thrift.protocol import *

import hbase_connector as hb


In [9]:
hbase = hb.HbaseConnection("datanode1", 9090)

In [10]:
## Deletes all tables in databse (except for hbase internal tables)
#with hbase as c:
#    for table in filter(lambda x: x.ns != b'hbase', tables):
#        if c.isTableEnabled(table):
#            c.disableTable(table)
#        c.deleteTable(table)

In [11]:
with hbase as c:
    c1 = TColumnFamilyDescriptor(name=b'c1')
    c2 = TColumnFamilyDescriptor(name=b'c2')
    name = TTableName(qualifier=b'demo_table')
    tt = TTableDescriptor(tableName=name, columns=[c1, c2])
    c.createTable(tt, None)

In [12]:
with hbase as c:
    tables = c.getTableNamesByNamespace('')
tables

[TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8AAA'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8AAQ'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8AAg'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8AAw'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2J1Y2tldF8ABA'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4X2tleXM'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAA'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAB'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAC'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcHJlZml4XzdfYnVja2V0XwAD'),
 TTableName(ns=b'default', qualifier=b'aGJhc2Vfc2FsbW9uZWxsYV9wb3NfcH

In [13]:
col1 = TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh')
put = TPut(row=b'hello row', columnValues=[col1])

with hbase as c:
    c.put(b'demo_table', put)

In [14]:
with hbase as c:
    v = c.get(b'demo_table', TGet(row=b'hello row'))
v

TResult(row=b'hello row', columnValues=[TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh', timestamp=1650984502807, tags=None, type=4)], stale=False, partial=False)

In [15]:
with hbase as c:
    col1 = TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh')
    put = TPut(row=b'anotha row', columnValues=[col1])
    
    col2 = TColumnValue(family=b'c2', qualifier=b'prefix4', value=b'adaaaaaa')
    put2 = TPut(row=b'not row3', columnValues=[col2])
    
    r = c.putMultiple(b'demo_table', [put, put2])
r

In [16]:
with hbase as c:
    r = c.getMultiple(b'demo_table', [TGet(row=b'hello row'), TGet(row=b'hello row3')])
r

[TResult(row=b'hello row', columnValues=[TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh', timestamp=1650984502807, tags=None, type=4)], stale=False, partial=False),
 TResult(row=None, columnValues=[], stale=False, partial=False)]

In [17]:
# filterBytes: https://hbase.apache.org/devapidocs/src-html/org/apache/hadoop/hbase/thrift2/ThriftUtilities.html#line.617
# So it's a protobuf serialized something

with hbase as c:
    s = TScan(filterString=b'PrefixFilter(\'hello\')')
    #s = TScan(reversed = True)
    scanner = c.openScanner(b'demo_table', tscan=s)
    
    readRows = None
    rows = []
    while readRows == None or len(readRows) == 1: 
        readRows = c.getScannerRows(scanner, numRows=1)
        rows.append(readRows)
    c.closeScanner(scanner)
    
rows

[[TResult(row=b'hello row', columnValues=[TColumnValue(family=b'c1', qualifier=b'prefix2', value=b'uhhhh', timestamp=1650984502807, tags=None, type=4)], stale=False, partial=False)],
 []]

In [18]:
with hbase as c:
    r = c.tableExists(TTableName(qualifier=b'nope'))
r

False

In [19]:
with hbase as c:
    r = c.tableExists(TTableName(qualifier=b'demo_table'))
r

True

In [20]:
with hbase as c:
    r = c.exists(b'demo_table', TGet(row=b'hello row'))
r

True